# MNIST Experiments

This notebook is created to experiment with [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset.

Plan:
- Dense network solution on CPU
- Dense network solution on GPU
- CNN+dense network solution on CPU
- CNN+dense network solution on GPU

Tools: Keras, PyTorch

In [1]:
import keras

Using TensorFlow backend.
/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [2]:
from keras.datasets import mnist
#from keras.datasets import fashion_mnist
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [53]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
#get train & tesr dataset
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
#(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [87]:
img_width = 28
img_height = 28
img_size = img_width * img_height # 784
train_img_number = train_data.shape[0]
test_img_number = test_data.shape[0]
classes_number = 10 # 10 digits 0-9
batch_size = 128
train_epochs = 10

In [88]:
# shape images
if K.image_data_format() == 'channels_first':
    train_data = train_data.reshape(train_img_number, 1, img_width, img_height)
    test_data = test_data.reshape(test_img_number, 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else: # channels_last
    train_data = train_data.reshape(train_img_number, img_width, img_height, 1)
    test_data = test_data.reshape(test_img_number, img_width, img_height, 1)
    input_shape = (img_width, img_height, 1)

In [89]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= 255
test_data /= 255

In [90]:
print('train shape', train_data.shape)

train shape (60000, 28, 28, 1)


In [91]:
train_labels = keras.utils.to_categorical(train_labels, classes_number)
test_labels = keras.utils.to_categorical(test_labels, classes_number)

In [127]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape)) # 32x28x28
model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) # 64x28x28
model.add(MaxPooling2D(pool_size=(2,2))) # 64x14x14
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu')) # 128
model.add(Dropout(0.50))
model.add(Dense(classes_number, activation='softmax')) # 10

model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_51 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_37 (Dropout)         (None, 128)             

In [121]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.adadelta(), #keras.optimizers.adadelta(),
    metrics=['accuracy']
)

In [122]:
print(test_data.shape)
print(test_labels.shape)

(10000, 28, 28, 1)
(10000, 10)


In [123]:
train_epochs = 10

model.fit(
    train_data, train_labels, 
    batch_size=batch_size, 
    epochs=train_epochs, 
    verbose=1, 
    validation_data=(test_data, test_labels)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 30us/step - loss: 1.7500 - accuracy: 0.3865 - val_loss: 1.4751 - val_accuracy: 0.4943
Epoch 2/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.3747 - accuracy: 0.5370 - val_loss: 1.2651 - val_accuracy: 0.5822
Epoch 3/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.2352 - accuracy: 0.5888 - val_loss: 1.1862 - val_accuracy: 0.6143
Epoch 4/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.1681 - accuracy: 0.6136 - val_loss: 1.1293 - val_accuracy: 0.6361
Epoch 5/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.1276 - accuracy: 0.6267 - val_loss: 1.1006 - val_accuracy: 0.6477
Epoch 6/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.1009 - accuracy: 0.6351 - val_loss: 1.0772 - val_accuracy: 0.6507
Epoch 7/10
60000/60000 [==============================] - 2s 28us/st

In [85]:
score = model.evaluate(test_data, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03707205359062646
Test accuracy: 0.9879999756813049


# Dog CNN with Keras

## Dependancies

In [4]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob

In [5]:
dog_classes = 133

In [6]:
def load_dataset(path, classes):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), classes)
    return files, targets

## Loading dataset

In [7]:
train_files, train_labels = load_dataset('dogImages/train', dog_classes)
valid_files, valid_labels = load_dataset('dogImages/valid', dog_classes)
test_files, test_labels = load_dataset('dogImages/test', dog_classes)

In [8]:
dog_labels = [item[16:-1] for item in sorted(glob('dogImages/train/*/'))]

In [9]:
img_size = (224, 224)

## Preprocessing images

In [10]:
from keras.preprocessing import image
from tqdm import tqdm_notebook as tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [11]:
def path_to_tensor(path):
    # load image
    img = image.load_img(path, target_size=img_size)
    # convert to 3D tensor
    x = image.img_to_array(img)
    # convert into 4D tensor at beginning of new axis
    return np.expand_dims(x, axis=0)    

In [12]:
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [13]:
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


## CNN dependancies

In [63]:
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

In [74]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, activation='relu', input_shape=train_tensors.shape[1:]))
model.add(BatchNormalization(axis=3, scale=False))
model.add(MaxPool2D(4))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(BatchNormalization(axis=3, scale=False))
model.add(MaxPool2D(4))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization(axis=3, scale=False))
model.add(MaxPool2D(4))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Flatten())
model.add(Dropout(0.25))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.50))

model.add(Dense(dog_classes, activation='softmax'))

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 223, 223, 16)      208       
_________________________________________________________________
batch_normalization_10 (Batc (None, 223, 223, 16)      48        
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 55, 55, 16)        0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 55, 55, 16)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 54, 54, 32)        2080      
_________________________________________________________________
batch_normalization_11 (Batc (None, 54, 54, 32)        96        
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 13, 13, 32)      

In [75]:
import keras

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.adadelta(), #keras.optimizers.adadelta(),
    metrics=['accuracy']
)

In [76]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    filepath='keras_dog_breed_classifier.hdf5', 
    save_best_only=True,
    verbose=1
)

In [77]:
epochs = 20
batch_size = 20

model.fit(
    train_tensors, train_labels, 
    validation_data=(valid_tensors, valid_labels), 
    epochs=epochs, 
    batch_size=batch_size, 
    callbacks=[checkpointer],
    verbose=1
)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 26s 4ms/step - loss: 5.1426 - accuracy: 0.0251 - val_loss: 6.5611 - val_accuracy: 0.0144

Epoch 00001: val_loss improved from inf to 6.56115, saving model to keras_dog_breed_classifier.hdf5
Epoch 2/20
6680/6680 [==============================] - 24s 4ms/step - loss: 4.5577 - accuracy: 0.0455 - val_loss: 5.8987 - val_accuracy: 0.0275

Epoch 00002: val_loss improved from 6.56115 to 5.89871, saving model to keras_dog_breed_classifier.hdf5
Epoch 3/20
6680/6680 [==============================] - 24s 4ms/step - loss: 4.2829 - accuracy: 0.0672 - val_loss: 7.0065 - val_accuracy: 0.0527

Epoch 00003: val_loss did not improve from 5.89871
Epoch 4/20
6680/6680 [==============================] - 24s 4ms/step - loss: 4.1732 - accuracy: 0.0717 - val_loss: 5.2462 - val_accuracy: 0.0491

Epoch 00004: val_loss improved from 5.89871 to 5.24616, saving model to keras_dog_breed_classifier.hdf5
Epoch 5/20

In [78]:
model.load_weights('keras_dog_breed_classifier.hdf5')
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]
# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_labels, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 11.4833%


In [18]:
np.argmax(test_labels, axis=1)

array([ 56,  58, 114,  18,  67,  20,  42,  51,  96,  50,   1,  18,  74,
       132,  70,  13, 122,  31,  90,  41,  97,  68,  55,  94,  35, 124,
        18,  18,  91, 112,   4, 108,  28,  22, 105,  79,  34,  38,   3,
       117,  80, 129,  41,  26,  60,   1, 100,  89,  87,   7,  32,  66,
        32,  13,  34, 118, 101, 103,  29,  96,  10, 102,  33,  95,  99,
        27, 132, 121,  42,  37,  87,  57, 100,  58,  70,  60,  29, 123,
        37,  67, 127, 102,  39,  81,  92,  86,  64,  45,   5,  36,  93,
        14, 130,  69,  21,  19, 128,  69,  34, 108,   7,  10,  58,  23,
        71, 114,  51,  56,  83,  86,  99,  60,  43,  11,  70,  10, 118,
        82,  27,  43,  75,   0, 131,   0,  36, 109,  80,  94,  43,  20,
        47, 117,  26, 117,  69,  53,  19,  55,  53,  49,  64,  74,  47,
        55,   4,   2,  66,  43,  87,   6, 124, 100,  15,  82,  91,   1,
        67,   9,  85, 112,  97,  28, 122,  87,  34,   0, 130,  97, 115,
        90,  71, 120, 117,  59,  35,  72, 119,  35, 111,  62,  4